In [1]:
def crop_center(img, cropsize, shift):
        (cropx, cropy, cropz) = cropsize
        (shiftx, shifty, shiftz) = shift
        y,x,z = img.shape
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)
        startz = z//2-(cropz//2)      
        return img[starty+shifty:starty+cropy+shifty,startx+shiftx:startx+cropx+shiftx, startz+shiftz:startz+cropz+shiftz]

In [8]:
import torch
import numpy as np
import os
from torch.utils.data import TensorDataset, DataLoader
import util

def make_blocks_vectorized(x,d):
    
    # d: chuck size
    p,m,n = x.shape
    return x.reshape(-1,m//d,d,n//d,d).transpose(1,3,0,2,4).reshape(-1,p,d,d) # shape: number of chuncks, x_dimention, d, d

def unmake_blocks_vectorized(x,d,m,n):    
    return np.concatenate(x).reshape(m//d,n//d,d,d).transpose(0,2,1,3).reshape(m,n)

def path_prep_data(rootpath, patient_ID):
    patient_ID = str(patient_ID)

    idx_path = os.path.join(rootpath ,"p" + patient_ID + "t1", "p" + patient_ID + "t1ct1_raw")
    roi_path = os.path.join(rootpath ,"p" + patient_ID + "t1", "p" + patient_ID + "t1ct1_raw_roi")

    return  idx_path, roi_path


idx_data = []
roi_data = []


patient_ID_list = [1,2,3,4]

rootpath = os.path.join(os.path.dirname(os.getcwd()),'python')
for patient_ID in  patient_ID_list:
    idx_path, roi_path = path_prep_data(rootpath, patient_ID)

    idx = np.load(idx_path + '.npy')
    roi = np.load(roi_path + '.npy').transpose(3,0,1,2)

    idx = (idx > 200)
    idx = idx.astype(int)

    # roi_new = np.zeros([roi.shape[0],200,128,256])
    roi_new = np.zeros([roi.shape[0],200,128,256])
    idx = crop_center(idx, (128, 200, 256),(0, 0, -30))
    for i in range(roi.shape[0]):
        roi_new[i] = crop_center(roi[i], (128, 200, 256),(0, 0, -30))

    

    

    
    d=128
    roi_chuncks = []
    idx_chuncks = make_blocks_vectorized(idx,d)
    for i  in range(roi_new.shape[0]):
        roi_chuncks.append(make_blocks_vectorized(roi_new[i],d))

    roi_chuncks = np.array(roi_chuncks)

    idx_data.append(idx_chuncks)
    roi_data.append(roi_chuncks.transpose(1,0,2,3,4))

    # use the whole volume to train instead of chunks
    # idx_data.append(idx)
    # roi_data.append(roi_new)



idx_data = np.concatenate(idx_data, 0)
roi_data = np.concatenate(roi_data, 0)




x_data = torch.Tensor(np.stack([idx_data], 4)).permute(0,4,1,2,3)
#x_data = x_data.unsqueeze(2)
y_data = torch.Tensor(np.array(roi_data))
# y_data = y_data.unsqueeze(1)

final_dataset = TensorDataset(x_data,y_data) # create your datset
savepath = r"G:\My Drive\PETCT\python"
torch.save(final_dataset, os.path.join(savepath, 'data_roi', 'petctdataset_train_chunck.pth'))



idx_data = []
roi_data = []


idx_data_whole = []
roi_data_whole = []

patient_ID_list = [7,8]

rootpath = os.path.join(os.path.dirname(os.getcwd()),'python')
for patient_ID in  patient_ID_list:
    idx_path, roi_path = path_prep_data(rootpath, patient_ID)

    idx = np.load(idx_path + '.npy')
    roi = np.load(roi_path + '.npy').transpose(3,0,1,2)

    idx = (idx > 200)
    idx = idx.astype(int)

    

    roi_new = np.zeros([roi.shape[0],200,128,256])
    idx = crop_center(idx, (128, 200, 256),(0, 0, -30))
    # roi_new = roi
    for i in range(roi.shape[0]):
        roi_new[i] = crop_center(roi[i], (128, 200, 256),(0, 0, -30))


   
    
    idx_data_whole.append(idx)
    roi_data_whole.append(roi_new)

    
x_data_whole = torch.Tensor(np.stack([np.array(idx_data_whole)], 4)).permute(0,4,1,2,3)

y_data_whole = torch.Tensor(np.array(roi_data_whole))
# y_data_whole = y_data_whole.unsqueeze(1)
final_dataset_whole = TensorDataset(x_data_whole,y_data_whole) # create your datset
torch.save(final_dataset_whole, os.path.join(savepath, 'data_roi', 'petctdataset_test_whole.pth'))